#### Processamento
Preparação dos dados para modelagem de clusterização

In [60]:
import numpy as np
import pandas as pd

In [61]:
# Carregar datasets
olist_orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
olist_order_items = pd.read_csv("../data/raw/olist_order_items_dataset.csv")
olist_customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")

- Join das tabelas
- Filtro de variáveis (apenas as que interessam)
- Construção de features:
    - faturamento total por cliente
    - quantidade total de pedidos por cliente

In [62]:
# Join das tabelas
customers_orders = olist_customers.merge(olist_orders, how="inner", on="customer_id")
customers_orders_items = customers_orders.merge(olist_order_items, how="inner", on="order_id" )

##### Faturamento por cliente

In [63]:
# Filtro de variáveis (quais colunas utilizar)
customers_orders_items.columns

# objetivo: faturamento e total de pedidos por consumidor
# customer_unique_id, order_id, order_status, price, freight_value

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [64]:
# Remove colunas
customers_orders_items.drop(columns=[
                                        i for  i in customers_orders_items.columns if i not in ['customer_unique_id', 'order_id', 'order_status', 'price', 'freight_value']
                                    ],
                            inplace = True )

In [65]:
# Imprime dataframe
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


In [66]:

customers_orders_items.order_status.unique()
# pedidos 'created' não fazem mais parte do df

array(['delivered', 'invoiced', 'shipped', 'processing', 'canceled',
       'unavailable', 'approved'], dtype=object)

In [67]:
# Filtro de registros
# Removendo 'canceled' e 'unavailable
customers_orders_items = customers_orders_items[(customers_orders_items.order_status != 'unavailable') &
                                                (customers_orders_items.order_status != 'canceled')
                                                ]

In [68]:
customers_orders_items

,customer_unique_id,order_id,order_status,price,freight_value
0,861eff4711a542e4b93843c6dd7febb0,00e7ee1b050b8499577073aeb2a297a1,delivered,124.99,21.88
1,290c77bc529b7ac935b93aa66c333dc3,29150127e6685892b6eab3eec79f59c7,delivered,289.00,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,139.94,17.79
3,259dac757896d24d7702b9acbbff3f3c,951670f92359f4fe4a63112aa7306eba,delivered,149.94,23.36
4,345ecd01c38d18a9036ed96c73b8d066,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,230.00,22.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,6760e20addcf0121e9d58f2f1ff14298,delivered,74.90,13.88
112646,d52a67c98be1cf6a5c84435bd38d095d,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,114.90,14.16
112647,e9f50caf99f032f0bf3c55141f019d99,fed4434add09a6f332ea398efd656a5c,delivered,37.00,19.04
112648,73c2643a0a458b49f58cea58833b192e,e31ec91cea1ecf97797787471f98a8c2,delivered,689.00,22.07


In [69]:
values_by_customer = customers_orders_items.groupby(by="customer_unique_id").sum()

In [70]:
values_by_customer

,price,freight_value
customer_unique_id,,
0000366f3b9a7992bf8c76cfdf3221e2,129.90,12.00
0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,8.29
0000f46a3911fa3c0805444483337064,69.00,17.22
0000f6ccb0745a6a4b88665a16c9f078,25.99,17.63
0004aac84e0df4da2b147fca70cf8255,180.00,16.89
...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,497.42
fffea47cd6d3cc0a88bd621562a9d061,64.89,19.69
ffff371b4d645b6ecea244b27531430a,89.90,22.56


In [71]:
olist_customers.customer_unique_id.unique().shape

(96096,)

In [72]:
values_by_customer["total_value"] = values_by_customer["price"] + values_by_customer["freight_value"]

In [73]:
values_by_customer

,price,freight_value,total_value
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,129.90,12.00,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,18.90,8.29,27.19
0000f46a3911fa3c0805444483337064,69.00,17.22,86.22
0000f6ccb0745a6a4b88665a16c9f078,25.99,17.63,43.62
0004aac84e0df4da2b147fca70cf8255,180.00,16.89,196.89
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,1570.00,497.42,2067.42
fffea47cd6d3cc0a88bd621562a9d061,64.89,19.69,84.58
ffff371b4d645b6ecea244b27531430a,89.90,22.56,112.46


In [74]:
values_by_customer.describe()

,price,freight_value,total_value
count,94983.000000,94983.000000,94983.000000
mean,142.071747,23.595025,165.666772
std,216.074999,22.867488,226.726478
min,0.850000,0.000000,9.590000
25%,47.900000,14.090000,63.100000
50%,89.890000,17.600000,107.900000
75%,155.000000,25.540000,182.930000
max,13440.000000,1794.960000,13664.080000


In [75]:
values_by_customer.drop(columns=["price", "freight_value"], inplace=True)

In [76]:
values_by_customer

,total_value
customer_unique_id,
0000366f3b9a7992bf8c76cfdf3221e2,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,27.19
0000f46a3911fa3c0805444483337064,86.22
0000f6ccb0745a6a4b88665a16c9f078,43.62
0004aac84e0df4da2b147fca70cf8255,196.89
...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2067.42
fffea47cd6d3cc0a88bd621562a9d061,84.58
ffff371b4d645b6ecea244b27531430a,112.46


In [77]:
values_by_customer = values_by_customer.reset_index(level=0)
values_by_customer

,customer_unique_id,total_value
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19
2,0000f46a3911fa3c0805444483337064,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,43.62
4,0004aac84e0df4da2b147fca70cf8255,196.89
...,...,...
94978,fffcf5a5ff07b0908bd4e2dbc735a684,2067.42
94979,fffea47cd6d3cc0a88bd621562a9d061,84.58
94980,ffff371b4d645b6ecea244b27531430a,112.46
94981,ffff5962728ec6157033ef9805bacc48,133.69


##### Quantidade de pedidos por cliente

In [78]:
# Filtro de registros
# Removendo 'canceled' e 'unavailable
orders_by_customers = customers_orders[(customers_orders.order_status != 'unavailable') &
                                            (customers_orders.order_status != 'canceled') &
                                            (customers_orders.order_status != 'created')
                                        ]

In [79]:
orders_by_customers = orders_by_customers.groupby(by="customer_unique_id").count()["order_id"]

In [80]:
orders_by_customers

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    1
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Name: order_id, Length: 94986, dtype: int64

In [81]:
orders_by_customers = orders_by_customers.reset_index(level=0)
orders_by_customers

,customer_unique_id,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...
94981,fffcf5a5ff07b0908bd4e2dbc735a684,1
94982,fffea47cd6d3cc0a88bd621562a9d061,1
94983,ffff371b4d645b6ecea244b27531430a,1
94984,ffff5962728ec6157033ef9805bacc48,1


In [82]:
customers_info = values_by_customer.merge(orders_by_customers, how="inner", on="customer_unique_id")
customers_info

,customer_unique_id,total_value,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19,1
2,0000f46a3911fa3c0805444483337064,86.22,1
3,0000f6ccb0745a6a4b88665a16c9f078,43.62,1
4,0004aac84e0df4da2b147fca70cf8255,196.89,1
...,...,...,...
94978,fffcf5a5ff07b0908bd4e2dbc735a684,2067.42,1
94979,fffea47cd6d3cc0a88bd621562a9d061,84.58,1
94980,ffff371b4d645b6ecea244b27531430a,112.46,1
94981,ffff5962728ec6157033ef9805bacc48,133.69,1


In [83]:
customers_info.rename(columns={"order_id": "qnt_orders"}, inplace=True)

In [84]:
customers_info

,customer_unique_id,total_value,qnt_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19,1
2,0000f46a3911fa3c0805444483337064,86.22,1
3,0000f6ccb0745a6a4b88665a16c9f078,43.62,1
4,0004aac84e0df4da2b147fca70cf8255,196.89,1
...,...,...,...
94978,fffcf5a5ff07b0908bd4e2dbc735a684,2067.42,1
94979,fffea47cd6d3cc0a88bd621562a9d061,84.58,1
94980,ffff371b4d645b6ecea244b27531430a,112.46,1
94981,ffff5962728ec6157033ef9805bacc48,133.69,1


In [85]:
customers_info.describe()

,total_value,qnt_orders
count,94983.000000,94983.000000
mean,165.666772,1.033859
std,226.726478,0.210811
min,9.590000,1.000000
25%,63.100000,1.000000
50%,107.900000,1.000000
75%,182.930000,1.000000
max,13664.080000,16.000000


In [86]:
customers_info.to_csv("../data/processed/customers_info.csv")